# Analyse

In [1]:
from dotenv import load_dotenv
import os
import pandas
from sqlalchemy import create_engine, text
from tqdm.notebook import tqdm_notebook

## Utils

In [2]:
def add_zero(number : int):
    if number > 9:
        return f"{number}"
    return f"0{number}"

## Create Engine

In [3]:
load_dotenv()

POSTGRESQL_URL = os.getenv("POSTGRESQL_URL")
engine = create_engine(POSTGRESQL_URL)

## Best Week - Month - Quarter

In [4]:
COLUMNS = ["quarter", "month", "week"]

### Get Weeks - Months - Quarter

In [5]:
for column in COLUMNS:
    data = []

    with engine.connect() as con:
        statement = f'SELECT DISTINCT year, {column} FROM public."History"'
        rows = con.execute(text(statement))
        for row in rows:
            year = row[0]
            time = row[1]
            data.append({
                column: time,
                "year": year
            })
        con.close()
    # Data Frame
    data_frame = pandas.DataFrame(data)
    data_frame.to_csv(f"./archives/{column}s.csv", index=False, header = True)

### Get Symbols

In [6]:
symbols = []

with engine.connect() as con:
    statement = 'SELECT DISTINCT symbol FROM public."History"'
    rows = con.execute(text(statement))
    for row in rows:
        symbol = row[0]
        symbols.append(symbol)
    con.close()

print(len(symbols))

with open('./archives/symbols.txt', 'w') as fp:
    fp.write('\n'.join(symbols))

1583


### Get Unique Symbols by Weeks - Months - Quarters

In [7]:
for column in COLUMNS:
    data_frame = pandas.read_csv(f"./archives/{column}s.csv")
    data = data_frame.to_dict("records")

    symbols_by_times = []

    for item in tqdm_notebook(data):
        time = item.get(column)
        year = item.get("year")
        symbols_by_time = []
        with engine.connect() as con:
            statement = f'SELECT DISTINCT symbol FROM public."History" WHERE {column} = {time} AND year = {year}'
            rows = con.execute(text(statement))
            for row in rows:
                symbol = row[0]
                symbols_by_time.append(symbol)
            con.close()
        symbols_by_times.append({
            "year": year,
            column: time,
            "symbols": symbols_by_time
        })
        folder = f'./archives/{column}s/{year}/{add_zero(time)}'
        is_exist = os.path.exists(folder)
        if not is_exist:
            os.makedirs(folder)
        with open(f'{folder}/symbols.txt', 'w') as fp:
            fp.write('\n'.join(symbols_by_time))

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/1167 [00:00<?, ?it/s]

### Get Changes by Weeks

In [8]:
for column in COLUMNS:
    time_frame = pandas.read_csv(f"./archives/{column}s.csv")
    data = time_frame.to_dict("records")
    for item in tqdm_notebook(data):
        year = item.get("year")
        time = item.get(column)
        symbols_file = open(f"./archives/{column}s/{year}/{add_zero(time)}/symbols.txt", "r")
        symbols = list(filter(lambda item: item != "", symbols_file.read().split("\n")))
        changes_by_time = []
        for symbol in symbols:
            with engine.connect() as con:
                first_subquery = f'SELECT MIN(full_date) FROM public."History" WHERE {column} = {time} AND year = {year} AND symbol LIKE \'{symbol}\''
                last_subquery = f'SELECT MAX(full_date) FROM public."History" WHERE {column} = {time} AND year = {year} AND symbol LIKE \'{symbol}\''
                # Close
                close_statement = f'SELECT close FROM public."History" WHERE (full_date LIKE ({first_subquery}) OR full_date LIKE ({last_subquery})) AND {column} = {time} AND year = {year} AND symbol LIKE \'{symbol}\' ORDER BY full_date ASC'
                close_cursor = con.execute(text(close_statement))
                close_records = close_cursor.fetchall()
                first_close = 0
                last_close = 0
                for idx, row in enumerate(close_records):
                    if idx == 0:
                        first_close = row[0]
                    else:
                        if close_cursor.rowcount > 1:
                            last_close = row[0]
                        else:
                            last_close = first_close
                # Change
                change = round((last_close - first_close) / first_close, 2)
                change_percentage = round(change * 100)
                changes_by_time.append({
                    "symbol": symbol,
                    "first": first_close,
                    "last": last_close,
                    "change": change,
                    "change_percentage": change_percentage
                })
                con.close()
        changes_by_time_data_frame = pandas.DataFrame(changes_by_time)
        sorted_changes_by_time_data_frame = changes_by_time_data_frame.sort_values(ascending=False, by=["change", "symbol"])
        sorted_changes_by_time_data_frame.to_csv(f'./archives/{column}s/{year}/{add_zero(time)}/changes.csv', index=False, header = True)

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/1167 [00:00<?, ?it/s]